### Mount Bucket

In [0]:
from pyspark.sql.functions import *
import urllib


aws_keys_df = spark.read.format("csv")\
.option("header", "true")\
.option("sep", ",")\
.load("/FileStore/tables/authentication_credentials.csv")

ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

AWS_S3_BUCKET = "user-0ea903d23769-bucket"
MOUNT_NAME = "/mnt/0ea903d23769-bucket"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

### Read Topic Contents Into Dataframes

In [0]:
def read_data(spark, file_type, infer_schema, location):
    return spark.read.format(file_type) \
                    .option("inferSchema", infer_schema) \
                    .load(location)

file_type = "json"
infer_schema = "true"

location_pin = "/mnt/0ea903d23769-bucket/topics/0ea903d23769.pin/partition=0/*.json"
location_geo = "/mnt/0ea903d23769-bucket/topics/0ea903d23769.geo/partition=0/*.json"
location_user = "/mnt/0ea903d23769-bucket/topics/0ea903d23769.user/partition=0/*.json"

df_pin_dirty = read_data(spark, file_type, infer_schema, location_pin)
df_geo_dirty = read_data(spark, file_type, infer_schema, location_geo)
df_user_dirty = read_data(spark, file_type, infer_schema, location_user)

### Download Dirty Data

In [0]:
display(df_pin_dirty)
display(df_geo_dirty)
display(df_use_dirty)

### Create Spark Session

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, TimestampType, DateType

spark = SparkSession.builder.appName("DF Cleaning").getOrCreate()

### Clean df_pin

In [0]:
df_pin = df_pin_dirty

# Replace empty entries and entries with no relevant data in each column with Nones
df_pin = df_pin.replace(['', ' ', 'NULL', 'null'], [None] * 4)
df_pin = df_pin.withColumn("description", when(col("description") == "No description available Story format", None).otherwise(col("description")))
df_pin = df_pin.withColumn("follower_count", when(col("follower_count") == "User Info Error", None).otherwise(col("follower_count")))
df_pin = df_pin.withColumn("image_src", when(col("image_src") == "Image src error.", None).otherwise(col("image_src")))
df_pin = df_pin.withColumn("poster_name", when(col("poster_name") == "User Info Error", None).otherwise(col("poster_name")))
df_pin = df_pin.withColumn("tag_list", when(col("tag_list") == "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", None).otherwise(col("tag_list")))
df_pin = df_pin.withColumn("title", when(col("title") == "No Title Data Available", None).otherwise(col("title")))

# Transform follower_count to ensure every entry is a number and data type is an int
# Remove any non-numeric characters (like 'k' in '136k') and then convert to integer
df_pin = df_pin.withColumn("follower_count", regexp_replace(col("follower_count"), "[^0-9]", ""))
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast(IntegerType()))

# Ensure that each column containing numeric data has a numeric data type
df_pin = df_pin.withColumn("downloaded", col("downloaded").cast(IntegerType()))

# Clean the data in the save_location column to include only the save location path
df_pin = df_pin.withColumn("save_location", regexp_replace(col("save_location"), "Local save in ", ""))

# Rename the index column to ind
df_pin = df_pin.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns
df_pin = df_pin.select(["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"])


display(df_pin)

### Clean df_geo

In [0]:
df_geo = df_geo_dirty

# Create a new column 'coordinates' containing an array of latitude and longitude
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop the latitude and longitude columns
df_geo = df_geo.drop("latitude", "longitude")

# Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn("timestamp", col("timestamp").cast(TimestampType()))

# Reorder the DataFrame columns
df_geo = df_geo.select(["ind", "country", "coordinates", "timestamp"])

display(df_geo)

### Clean df_user

In [0]:
df_user = df_user_dirty

# Create a new column 'user_name' by concatenating 'first_name' and 'last_name'
df_user = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# Drop the 'first_name' and 'last_name' columns
df_user = df_user.drop("first_name", "last_name")

# Convert the 'date_joined' column from a string to a timestamp data type
df_user = df_user.withColumn("date_joined", col("date_joined").cast(TimestampType()))

# Reorder the DataFrame columns
df_user = df_user.select(["ind", "user_name", "age", "date_joined"])

display(df_user)